In [1]:
import numpy as np
import gym
import k3d
from ratelimiter import RateLimiter
from k3d.platonic import Cube


env = gym.make('CartPole-v0')
observation = env.reset()
O = []

for t in range(500):
    #env.render()
    O.append(observation)
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    if observation[0] > 5.0 or observation[0] < -5.0:
        break

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.


In [2]:
O = np.array(O)

In [3]:
plot = k3d.plot(grid_auto_fit=False, camera_auto_fit=False, grid=(-5,-3,-3,5,3,3))

In [4]:
joint_positions = np.vstack([O[:,0], np.zeros(np.shape(O[:,0])), np.zeros(np.shape(O[:,0]))]).T
pole_positions = joint_positions + np.vstack([np.sin(O[:,2]), np.zeros(np.shape(O[:,2])), np.cos(O[:,2])]).T
joint_positions = np.float32(joint_positions)
pole_positions = np.float32(pole_positions)

cart = Cube(origin=joint_positions[0], size=0.1).mesh
cart.scaling = [1, 0.5, 1]
joint = k3d.points(joint_positions[0], point_size=0.2)
pole = k3d.line(vertices=np.array([joint_positions[0], pole_positions[0]]), shader='simple', color=0xff0000)
box = cart.vertices

plot += pole + cart

In [5]:
plot.display()

Output()

In [6]:
rate_limiter = RateLimiter(max_calls=1, period=1/30)

for i in range(np.shape(O)[0]):
    with rate_limiter:
        cart.vertices = box + joint_positions[i] 
        joint.positions = joint_positions[i]
        pole.vertices = [joint_positions[i], pole_positions[i]]